# Base Model Creation

In [1]:
from dataclasses import dataclass
from pathlib import Path
from box import ConfigBox

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    artifact_dir:Path
    params:ConfigBox
    base_model_path:Path
    updated_base_model_path:Path

In [2]:
from DeepClassifier.constants import CONFIG_FILEPATH, PARAMS_FILEPATH
from DeepClassifier.utils import read_yaml_file, make_directories
from pathlib import Path
from DeepClassifier.entity import DataIngestionConfig

class ConfigurationManager:
    def __init__(
            self,
            config_filepath:Path=CONFIG_FILEPATH,
            param_filepath:Path=PARAMS_FILEPATH,
        ):
        self.config = read_yaml_file(config_filepath)
        self.param = read_yaml_file(param_filepath)
        
        make_directories([self.config.root_artifact_dir])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        data_ingestion_config_info = self.config.data_ingestion
        
        artifact_dir = data_ingestion_config_info.artifact_dir

        make_directories([artifact_dir])

        data_ingestion_config = DataIngestionConfig(
                                    artifact_dir=artifact_dir,
                                    source_download_url=data_ingestion_config_info.source_download_url,
                                    downloaded_data_file_path=data_ingestion_config_info.downloaded_data_file_path,
                                    unzipped_data_dir=data_ingestion_config_info.unzipped_data_dir
                                )

        return data_ingestion_config


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        prepare_base_model_config_info = self.config.prepare_base_model

        artifact_dir = prepare_base_model_config_info.artifact_dir

        make_directories([artifact_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
                                        artifact_dir=Path(artifact_dir),
                                        params=self.param,
                                        base_model_path=Path(prepare_base_model_config_info.base_model_path),
                                        updated_base_model_path=Path(prepare_base_model_config_info.updated_base_model_path)
                                    )

        return prepare_base_model_config



In [3]:
import tensorflow as tf
from DeepClassifier.utils import read_yaml_file

class PrepareBaseModel:
    def __init__(self, config=PrepareBaseModelConfig) -> None:
        self.config=config

    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

    def get_base_model(self):
        self.base_model = tf.keras.applications.vgg16.VGG16(
                            input_shape=self.config.params.IMAGE_SHAPE,
                            include_top=self.config.params.INCLUDE_TOP,
                            weights=self.config.params.WEIGHTS
                        )

        base_model_path = self.config.base_model_path        

        self.save_model(base_model_path, self.base_model)

    
    @staticmethod
    def _prepare_full_model(base_model, classes, freeze_all, freeze_till, learning_rate) -> tf.keras.models.Model:
        if freeze_all:
            for layer in base_model.layers:
                layer.trainable = False

        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in base_model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_layer = tf.keras.layers.Flatten()(base_model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation='softmax'
        )(flatten_layer)

        full_model = tf.keras.models.Model(
            inputs=base_model.input,
            outputs=prediction
        )

        full_model.compile(
            loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            metrics=['accuracy']
        )

        return full_model


    def update_base_model(self):
        self.final_model = self._prepare_full_model(
                                base_model=self.base_model,
                                classes=self.config.params.CLASSES,
                                freeze_all=True,
                                freeze_till=None,
                                learning_rate=self.config.params.LEARNING_RATE
                            )

        
        self.save_model(self.config.updated_base_model_path, self.final_model)







In [4]:
import os
os.chdir('..')
os.getcwd()

'd:\\Full Stack Data Science\\Python Project\\DL\\Current Batch\\Deep_CNN_Classifier'

In [5]:
try:
    config= ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model= PrepareBaseModel(prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise (e)

In [6]:
m = tf.keras.models.load_model(r'D:\Full Stack Data Science\Python Project\DL\Current Batch\Deep_CNN_Classifier\artifacts\prepare_base_model\base_model_updated.h5')
m.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     